# Sketch Classification

## Load dataset
Loading dataset from local folder. The *load_sketches* returns shuffled data with labels, splitted for training and test.

In [11]:
import load_sketches as ld

In [12]:
train_set, test_set, train_labels, test_labels, label_strings = ld.load_sketches(0.9,"sketches/")

## Set up model

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [17]:
model = Sequential()
#model.add
#model.add

In [18]:
#model.compile()

## Train model

In [ ]:
#model.fit()